In [ ]:
import os
os.environ["GROQ_API_KEY"] = ""

In [ ]:
import os
os.environ["ACTIVELOOP_TOKEN"] = "<>"

In [1]:
from langchain_groq import ChatGroq
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.vectorstores import DeepLake
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

# Initialize Groq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.7
)

# 2️⃣ Initialize embeddings
embeddings = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

# 3️⃣ Create documents
texts = [
    "Napoleon Bonaparte was born on 15 August 1769.",
    "Louis XIV was born on 5 September 1638."
]

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=0
)

docs = text_splitter.create_documents(texts)

# 4️⃣ Create DeepLake vector store
db = DeepLake(
    dataset_path="./deeplake_dataset",
    embedding=embeddings
)

db.add_documents(docs)

# 5️⃣ Create retriever
retriever = db.as_retriever()

# 6️⃣ Create prompt
prompt = ChatPromptTemplate.from_template(
    """Answer the question based only on the context below:

{context}

Question: {question}
"""
)

# 7️⃣ Build modern RAG pipeline
rag_chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
)

# 8️⃣ Ask question
response = rag_chain.invoke("When was Napoleon born?")

print(response.content)


C:\Users\Chala\OneDrive\Desktop\LangChain01\langchain_env\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\Chala\OneDrive\Desktop\LangChain01\langchain_env\lib\site-packages\deeplake\util\check_latest_version.py:32: UserWarning: A newer version of deeplake (4.5.2) is available. It's recommended that you update to the latest version using `pip install -U deeplake`.
  warnings.warn(
C:\Users\Chala\OneDrive\Desktop\LangChain01\langchain_env\lib\site-packages\huggingface_hub\file_download.py:130: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Chala\.cache\huggingface\hub\models--sentence-transformers--all-MiniLM-L6-v2. Caching files will still work but in a degraded version that might require more spa

Dataset(path='./deeplake_dataset', tensors=['text', 'metadata', 'embedding', 'id'])

  tensor      htype     shape     dtype  compression
  -------    -------   -------   -------  ------- 
   text       text      (2, 1)     str     None   
 metadata     json      (2, 1)     str     None   
 embedding  embedding  (2, 384)  float32   None   
    id        text      (2, 1)     str     None   
Napoleon Bonaparte was born on 15 August 1769.
